In [ ]:
from __future__ import division, print_function, absolute_import
import tensorflow as tf
import scipy.io as sio
import numpy as np
import glob
from time import time
from PIL import Image
import math
import matplotlib.pyplot as plt
from tensorflow.python import debug as tf_debug
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

CS_ratio = 10    # 4, 10, 25, 30,, 40, 50

if CS_ratio == 1:
    m = 10
elif CS_ratio == 10:
    m = 109
elif CS_ratio == 25:
    m = 272
elif CS_ratio == 40:
    m = 436
elif CS_ratio == 50:
    m = 545

block_size = 33
n = block_size*block_size
batch_size = 64
nrtrain = 88912
learning_rate = 0.0005

EpochNum = 300
layer = 41
Training_data_Name = 'Training_Data_Img91.mat'
Training_data = sio.loadmat(Training_data_Name)
Training_labels = Training_data['labels']

Test_Img = '../Test_Image'

Phi_data_Name = 'phi_Folder/phi_0_%d_1089.mat' % CS_ratio
Phi_data = sio.loadmat(Phi_data_Name)
Phi_input = Phi_data['phi'].transpose()


XX = Training_labels.transpose()
YY = np.dot(Phi_input.transpose(), XX)
YTY = np.dot(YY, YY.transpose())
CCC = np.dot(XX, YY.transpose())
Q_init = tf.constant(np.dot(CCC, np.linalg.inv(YTY)).transpose(), dtype=tf.float32)
del XX, YY, YTY, CCC, Training_data

X_input = tf.placeholder(tf.float32, [None, m])
X_output = tf.placeholder(tf.float32, [None, n])
X0 = tf.matmul(X_input, Q_init)
Y = X_input


def imread_CS_py(imgName):
    block_size = 33
    Iorg = np.array(Image.open(imgName), dtype='float32')
    [row, col] = Iorg.shape
    row_pad = block_size-np.mod(row,block_size)
    col_pad = block_size-np.mod(col,block_size)
    Ipad = np.concatenate((Iorg, np.zeros([row, col_pad])), axis=1)
    Ipad = np.concatenate((Ipad, np.zeros([row_pad, col+col_pad])), axis=0)
    [row_new, col_new] = Ipad.shape
    return [Iorg, row, col, Ipad, row_new, col_new]

def img2col_py(Ipad, block_size):
    [row, col] = Ipad.shape
    row_block = row/block_size
    col_block = col/block_size
    block_num = int(row_block*col_block)
    img_col = np.zeros([block_size**2, block_num])
    count = 0
    for x in range(0, row-block_size+1, block_size):
        for y in range(0, col-block_size+1, block_size):
            img_col[:, count] = Ipad[x:x+block_size, y:y+block_size].reshape([-1])
            count = count + 1
    return img_col

def col2im_CS_py(X_col, row, col, row_new, col_new):
    block_size = 33
    X0_rec = np.zeros([row_new, col_new])
    count = 0
    for x in range(0, row_new-block_size+1, block_size):
        for y in range(0, col_new-block_size+1, block_size):
            X0_rec[x:x+block_size, y:y+block_size] = X_col[:, count].reshape([block_size, block_size])
            count = count + 1
    X_rec = X0_rec[:row, :col]
    return X_rec

def psnr(img1, img2):
    img1.astype(np.float32)
    img2.astype(np.float32)
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0:
        return 100
    PIXEL_MAX = 255.0
    return 20 * math.log10(PIXEL_MAX / math.sqrt(mse))

filepaths = glob.glob(Test_Img + '/*.tif')
ImgNum = len(filepaths)
PSNR_All = np.zeros([1, ImgNum], dtype=np.float32)

In [ ]:
def softshrink(x, lambd):
    return tf.nn.relu(x - lambd) - tf.nn.relu(-1*x - lambd)

def conv_dict(Input, weight1, weight2):
    Output = tf.nn.conv2d(Input, weight1, strides=[1, 1, 1, 1], padding='SAME')
    Output = tf.nn.relu(Output)
    Output = tf.nn.conv2d(Output, weight2, strides=[1, 1, 1, 1], padding='SAME')
    return Output

def truncate(Input, weights, index):
    Input = tf.reshape(Input, shape=[-1, block_size, block_size, 1])
    dict_output = conv_dict(Input, weights['filter_1%d' %(index)], weights['filter_2%d' %(index)])
    shrink_output = softshrink( dict_output, weights['Lambda_%d' %(index)]  )
    final_output = conv_dict( shrink_output - dict_output, weights['filter_5%d' %(index)], weights['filter_6%d' %(index)])
    final_output = tf.reshape(final_output, shape=[-1, n])
    return final_output

def sym_op(Input, weights, index):
    Input = tf.reshape(Input, shape=[-1, block_size, block_size, 1])
    Output = conv_dict(Input, weights['filter_1%d' %(index)], weights['filter_2%d' %(index)])
    Output = conv_dict(Output, weights['filter_5%d' %(index)], weights['filter_6%d' %(index)])
    return Output - Input


def forwarding(Y, weights, layer):
    sym = []
    sol_set = []
    Y_Phi_T_G = tf.matmul(Y, Phi_T_G)
    X = X0
    M = tf.zeros_like(X)
    for i in range(layer-1):
        Z = tf.matmul(X, FC_X) + tf.matmul(M, FC_M(weights['Beta1_%d' %(i)])) + 10*Y_Phi_T_G
        X = Z + weights['Beta2_%d' %(i)]*truncate(Z, weights, i)
        M = weights['Beta3_%d' %(i)]*(Z - X)
        sym.append( sym_op(Z, weights, i) )
        sol_set.append(X)
    Z = tf.matmul(X, FC_X) + tf.matmul(M, FC_M(weights['Beta1_%d' %(layer-1)])) + 10*Y_Phi_T_G
    X = Z + weights['Beta2_%d' %(layer-1)]*truncate(Z, weights, layer-1)
    sym.append( sym_op(Z, weights, layer-1) )
    sol_set.append(X)
        
    return [sol_set, sym]

weights = {}
Phi_T_input = Phi_input.transpose()
PhiT_Phi = np.matmul(Phi_input, Phi_T_input).transpose()
g = np.transpose(np.linalg.inv(10*PhiT_Phi + 0.1*np.eye(n)))
Phi_T_G = tf.constant(np.matmul(Phi_T_input, g), dtype=tf.float32)
G = tf.constant(g, dtype=tf.float32)
FC_X = 0.1*G
def FC_M(beta):
    return (1/beta)*tf.eye(n) - G

for i in range(layer):
    temp = "filter_1%d" % (i)
    weights[temp] = tf.get_variable(shape=[3,3, 1,32], initializer=tf.contrib.layers.xavier_initializer_conv2d(), name="filter_1%d" % (i))
    temp = "filter_2%d" % (i)
    weights[temp] = tf.get_variable(shape=[3,3,32,32], initializer=tf.contrib.layers.xavier_initializer_conv2d(), name="filter_2%d" % (i))
    temp = "filter_5%d" % (i)
    weights[temp] = tf.get_variable(shape=[3,3,32,32], initializer=tf.contrib.layers.xavier_initializer_conv2d(), name="filter_5%d" % (i))
    temp = "filter_6%d" % (i)
    weights[temp] = tf.get_variable(shape=[3,3,32, 1], initializer=tf.contrib.layers.xavier_initializer_conv2d(), name="filter_6%d" % (i))
    temp = "Lambda_%d" % (i)
    weights[temp] = tf.Variable(0.02, name="Lambda_%d" % (i))
    temp = "Beta1_%d" % (i)
    weights[temp] = tf.Variable(0.3, name="Beta1_%d" % (i))
    temp = "Beta2_%d" % (i)
    weights[temp] = tf.Variable((2.0-(1.5*i/layer)), name="Beta2_%d" % (i))
for i in range(layer-1):
    temp = "Beta3_%d" % (i)
    weights[temp] = tf.Variable(0.2, name="Beta3_%d" % (i))


In [ ]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

saver = tf.train.Saver(tf.global_variables(), max_to_keep=1)

model_dir = 'Layer_%d_ratio_0_%d_ADMM_Net_Model' % (layer, CS_ratio)
output_file_name = "Log_output_%s.txt" % (model_dir)

result, sym = forwarding(Y, weights, layer)
loss_MSE = 0
for k in range(len(result)):
    loss_MSE += tf.reduce_mean(tf.square(result[k] - X_output))
loss_sym = 0
for k in range(len(sym)):
    loss_sym += tf.reduce_mean(tf.square(sym[k]))

loss_op =  loss_MSE + 0.01*loss_sym
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss_op)
    
init = tf.global_variables_initializer()
sess = tf.Session(config=config)

sess.run(init)

In [ ]:
print("Layer = %d, ratio = %d, training start:" % (layer, CS_ratio))
min_loss = 1
rec_PSNR = 0
for epoch_i in range(0, EpochNum+1):
    start = time()
    randidx_all = np.random.permutation(nrtrain)
    for batch_i in range(nrtrain // batch_size):
        randidx = randidx_all[batch_i*batch_size:(batch_i+1)*batch_size]
        batch_ys = Training_labels[randidx, :]
        batch_xs = np.matmul(batch_ys, Phi_input)
        feed_dict={X_input: batch_xs, X_output: batch_ys}
        _ = sess.run(optimizer, feed_dict=feed_dict)
       
    avg_PSNR = 0
    for img_no in range(ImgNum):
        imgName = filepaths[img_no]
        [Iorg, row, col, Ipad, row_new, col_new] = imread_CS_py(imgName)
        Icol = img2col_py(Ipad, block_size).transpose() /255.0
        Img_input = np.matmul(Icol, Phi_input)
        Prediction_value = sess.run(result[-1], feed_dict={X_input: Img_input})
        X_rec = col2im_CS_py(Prediction_value.transpose(), row, col, row_new, col_new)
        X_rec = np.clip(X_rec * 255, 0, 255, out = X_rec)
        avg_PSNR = avg_PSNR + psnr(Iorg, X_rec)
    avg_PSNR = avg_PSNR/11
    val_rec_PSNR = avg_PSNR
    output_data = "AVG PSNR is %.2f" % (avg_PSNR)
    print(output_data)
    if val_rec_PSNR > rec_PSNR:
        rec_PSNR = val_rec_PSNR
        saver.save(sess,'ckpt/%s/Saved_Model.cpkt' % (model_dir), global_step=epoch_i)
    end = time()
    print("Time cost %.2f" %(end - start))


In [ ]:
model_file = tf.train.latest_checkpoint( 'ckpt/%s/' % (model_dir) )
saver.restore(sess, model_file)
avg_PSNR = 0
avg_SSIM = 0
avg_running_time = 0

print("Layer = %d, ratio = %d, testing start:" % (layer, CS_ratio))
for img_no in range(ImgNum):
    imgName = filepaths[img_no]
    [Iorg, row, col, Ipad, row_new, col_new] = imread_CS_py(imgName)
    Icol = img2col_py(Ipad, block_size).transpose() /255.0
    Img_input = np.matmul(Icol, Phi_input)
    start = time()
    Prediction_value = sess.run(result[-1], feed_dict={X_input: Img_input})
    end = time()
    X_rec = col2im_CS_py(Prediction_value.transpose(), row, col, row_new, col_new)
    X_rec = np.clip(X_rec * 255, 0, 255, out = X_rec)
    rec_PSNR = psnr(Iorg, X_rec)
    temp = Iorg.shape
    rec_SSIM = sess.run(tf.image.ssim(tf.image.convert_image_dtype(tf.reshape(X_rec, shape=list(temp) + [1]), tf.float32), tf.reshape(Iorg, shape=list(temp) + [1]), max_val=255.0))
    output_data = "%s PSNR is %.2f, SSIM is %.2f, Run time is %.4f" % (imgName, rec_PSNR, rec_SSIM, (end - start) )
    print(output_data)
    img_rec_name = "%s_rec_PSNR_%.2f.png" % (imgName, rec_PSNR)
    x_im_rec = Image.fromarray(np.clip(X_rec * 255, 0, 255).astype(np.uint8))
    x_im_rec.save(img_rec_name)

    avg_running_time = avg_running_time + (end - start)
    avg_PSNR = avg_PSNR + rec_PSNR
    avg_SSIM = avg_SSIM + rec_SSIM
avg_PSNR = avg_PSNR/11
avg_SSIM = avg_SSIM/11
avg_running_time = avg_running_time/17
print("SET11 AVG PSNR is %.2f, AVG SSIM is %.2f, AVG running time is %.2f" % (avg_PSNR, avg_SSIM, avg_running_time))